In [22]:
import mlflow
import polars as pl
import pandas as pd
from pathlib import Path
from mlflow.tracking import MlflowClient


## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [8]:
# !mlflow --version

print(f"mlflow version: {mlflow.__version__}")

mlflow version: 2.22.0


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

In [14]:
# read the data from the URLs and dump them to a local folder

def download_data(url, filename):
    data = pl.read_parquet(url).write_parquet(f"data/{filename}.parquet")

download_data("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet", "green_tripdata_2023-01")
download_data("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet", "green_tripdata_2023-02")
download_data("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet", "green_tripdata_2023-03")

In [15]:
!python preprocess_data.py --raw_data_path data --dest_path ./output

In [18]:
# count the number of files in the output folder
files = list(Path('./output').glob('*'))
print(f"Number of files: {len(files)}")


Number of files: 4


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework). 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

In [21]:
!python train.py --data_path ./output

In [37]:
client = MlflowClient()

runs = client.search_runs(
    experiment_ids='0'
)

print(f"min_samples_split: {runs[0].data.params['min_samples_split']}")


min_samples_split: 2


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

In [41]:
# !mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root artifacts

print("mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root artifacts")

mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root artifacts


## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

In [43]:
!python hpo.py

🏃 View run redolent-hawk-18 at: http://127.0.0.1:5000/#/experiments/1/runs/bf31ff1ff7ea4ccf8cb8a1e17cde5615

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run nosy-zebra-612 at: http://127.0.0.1:5000/#/experiments/1/runs/20bdd89b6b3e47048856800083cd0dd5

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run traveling-lamb-896 at: http://127.0.0.1:5000/#/experiments/1/runs/361cce1f55644615a371c8b33ff911fc

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run aged-hawk-848 at: http://127.0.0.1:5000/#/experiments/1/runs/182dbff59ab64d73b201b8861159447e

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run placid-panda-53 at: http://127.0.0.1:5000/#/experiments/1/runs/e2d6f98f1e544ed2b47a9e4bd65057c1

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run brawny-fawn-150 at: http://127.0.0.1:5000/#

In [64]:
# Best RMSE: 5.335
mlflow.set_tracking_uri("http://127.0.0.1:5000")
client = MlflowClient()

best_rmse = client.search_runs(
    experiment_ids='1',
    order_by=['metrics.rmse ASC'],
    max_results=1
)[0].data.metrics['rmse']

print(f"The best RMSE is {best_rmse:.3f}.")


The best RMSE is 5.335.


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

In [67]:
!python register_model.py

2025/05/23 16:43:59 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
🏃 View run rumbling-ram-980 at: http://127.0.0.1:5000/#/experiments/2/runs/5348815269fb4213bb85c55c3ba79d4a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run serious-fawn-217 at: http://127.0.0.1:5000/#/experiments/2/runs/9102bcae6b9848d787b4162a5848f627
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run exultant-cod-631 at: http://127.0.0.1:5000/#/experiments/2/runs/bd9f1686da304b75876363a3426cc2fb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run angry-fawn-946 at: http://127.0.0.1:5000/#/experiments/2/runs/5b96bf09010647648c6a6bca248485cf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run awesome-foal-801 at: http://127.0.0.1:5000/#/experiments/2/runs/2f932aaa808f4416a6f6b78df60366db
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
The best run_id is 53

In [81]:
best_model_run_id = client.search_registered_models()[0].latest_versions[0].run_id

test_rmse = client.get_run(best_model_run_id).data.metrics['test_rmse']

print(f"The best test RMSE is {test_rmse:.3f}.")


The best test RMSE is 5.567.
